# Analyzing monochromator QE sweeps.
Craig Lage 21-Dec-23

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

from lsst.daf.butler import Butler
from lsst.summit.utils import BestEffortIsr
import lsst.afw.cameraGeom.utils as camGeomUtils
from lsst.ip.isr import IsrTask, IsrTaskConfig
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm

from lsst.summit.utils import quickSmooth
%matplotlib inline

from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask
from lsst.meas.algorithms.installGaussianPsf import InstallGaussianPsfTask

In [ ]:
def plotExp(exp):
    # Borrowed from summit utils
    data = quickSmooth(exp.image.array, 1)
    vmin = np.nanpercentile(data, 1)
    vmax = np.nanpercentile(data, 99)
    print(vmin, vmax)
    figure = plt.figure(figsize=(12,12))
    figure.clear()
    ax1 = figure.add_subplot(111)
    im1 = ax1.imshow(data, cmap='gray', origin='lower', vmin=vmin, vmax=vmax)
    ax1.tick_params(which="major", direction="in", top=True, right=True, labelsize=8)
    divider = make_axes_locatable(ax1)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(im1, cax=cax)
    plt.tight_layout()
    #plt.show()
    return figure

def repairCosmics(postIsr):
    # Borrowed from summit utils
    if postIsr.getPsf() is None:
        installPsfTask = InstallGaussianPsfTask()
        installPsfTask.run(postIsr)

    # TODO: try adding a reasonably wide Gaussian as a temp PSF
    # and then just running repairTask on its own without any
    # imChar. It should work, and be faster.
    repairConfig = CharacterizeImageTask.ConfigClass()
    repairConfig.doMeasurePsf = False
    repairConfig.doApCorr = False
    repairConfig.doDeblend = False
    repairConfig.doWrite = False
    repairConfig.repair.cosmicray.nCrPixelMax = 200000
    repairTask = CharacterizeImageTask(config=repairConfig)
    repairTask.repair.run(postIsr)
    return


In [ ]:
butler = Butler('/repo/embargo', collections=["LATISS/raw/all", "LATISS/calib", "LATISS/calib/unbounded"])
best_effort_isr = BestEffortIsr(embargo=True)
best_effort_isr.doWrite = False  # Don't write to butler database

In [ ]:
expId_filter = 2023121900074
expId_no_filter = 2023121900075

 ### Testing the ISR

In [ ]:
dataId_filter = {'exposure': expId_filter, 'detector': 0, "instrument": 'LATISS'}
exp_filter = best_effort_isr.getExposure(dataId_filter, skipCosmics=False)
dataId_no_filter = {'exposure': expId_no_filter, 'detector': 0, "instrument": 'LATISS'}
exp_no_filter = best_effort_isr.getExposure(dataId_no_filter, skipCosmics=False)
data_filter = quickSmooth(exp_filter.image.array, 1)
data_no_filter = quickSmooth(exp_no_filter.image.array, 1)
ratio = data_filter / data_no_filter

In [ ]:
vmin = np.nanpercentile(ratio, 1)
vmax = np.nanpercentile(ratio, 99)
figure = plt.figure(figsize=(12,12))
figure.clear()
ax1 = figure.add_subplot(111)
im1 = ax1.imshow(ratio, cmap='gray', origin='lower', vmin=vmin, vmax=vmax)
ax1.tick_params(which="major", direction="in", top=True, right=True, labelsize=8)
divider = make_axes_locatable(ax1)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im1, cax=cax)
plt.tight_layout()
print(np.median(ratio))

# Now process the measured images

In [ ]:
wave_step = 5 # Wavelength step in nm
min_wave = 500; max_wave = 750;
wave = min_wave
expId_filter = 2023121900024
expId_no_filter = 2023121900025

waves = []
transmittances = []
sigmas = []
while wave < max_wave + wave_step / 2.0:
    dataId_filter = {'exposure': expId_filter, 'detector': 0, "instrument": 'LATISS'}
    exp_filter = best_effort_isr.getExposure(dataId_filter, skipCosmics=False)
    dataId_no_filter = {'exposure': expId_no_filter, 'detector': 0, "instrument": 'LATISS'}
    exp_no_filter = best_effort_isr.getExposure(dataId_no_filter, skipCosmics=False)
    data_filter = quickSmooth(exp_filter.image.array, 1)
    data_no_filter = quickSmooth(exp_no_filter.image.array, 1)
    ratio = data_filter / data_no_filter
    transmittance = np.nanpercentile(ratio, 50) * 100.0
    sigma = (np.nanpercentile(ratio, 86) - np.nanpercentile(ratio, 14)) / 2.0 * 100.0
    waves.append(wave)
    transmittances.append(transmittance)
    sigmas.append(sigma)
    print(f"At {wave} nm, filter transmittance is {transmittance}%, sigma = {sigma}")
    wave += wave_step
    expId_filter += 2
    expId_no_filter += 2


# This cell gets the vendor supplied data.
This came from: https://github.com/lsst/obs_lsst_data/blob/main/latiss/transmission_filter/sdssr_65mm~empty/20221005T000000.ecsv

In [ ]:
import csv
official_waves = []
official_trans = []
with open('/home/c/cslage/u/AuxTel/filter_data/20221005T000000.ecsv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    counter = 0
    for row in spamreader:
        if counter > 28:
            # Rows before this don't contain the data
            official_waves.append(float(row[0]))
            official_trans.append(float(row[1]) * 100.0)
            print(row[0], float(row[1])*100.0)
        counter +=1

# Now plot the data

In [ ]:
plt.title("AuxTel SDSSr_65mm Transmission curve - 19-Dec-23")
plt.plot(waves, transmittances, marker='x', color='blue', label="Measured")
plt.plot(official_waves, official_trans, marker='+', color='red', label="Official")
plt.xlabel("Wavelength (nm)")
plt.ylabel("Transmittance (%)")
plt.xlim(500, 750)
plt.ylim(0,100.0)
plt.legend()
plt.savefig("/home/c/cslage/u/AuxTel/filter_data/SDSSr_65mm_Filter_Curve_19Dec23.png")

# I don't know why the command below didn't work:

In [ ]:
butler.get("transmission_filter", dataId={'instrument': 'LATISS', 'physical_filter': 'SDSSr_65mm~empty'})